In [1]:
import requests
from bs4 import BeautifulSoup
import json

def send_data(data_dict, form_id):
    ''' 
    Submits data to a Google Form.
    
    Parameters:
        data_dict (dict): A dictionary containing form data where keys are form item names and values are corresponding data.
        form_id (str): The ID of the Google Form to which the data will be submitted.
        
    Returns:
        bool: True if the data was successfully submitted, False otherwise.
    '''

    # Define the URLs for viewing and submitting the Google Form.
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    # Fetch the content of the view form page and extract the form data structure.
    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    # Prepare the data to be submitted to the Google Form.
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    # Submit the data to the Google Form using a POST request.
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok